# Projeto 3 - Ciência dos Dados

#  Projeto UFC

Nome: Victor Vergara Arcoverde de Albuquerque Cavalcanti

Nome: Edgard Neto

Nome: Gabriel Yamashita

Nome: Henrique Mualem Marti



  ___
## Objetivo:

### O objetivo desse projeto é fazer um machine learning para poder prever qual lutador ganhará uma luta do UFC baseado no seu histórico. Para isso serão usados os dados de todas as lutas do UFC (mais de 5 mil) a fim de descobrir quais os fatores dos lutadores que impactam mais no resultado das lutas. 

[Database utilizado](https://www.kaggle.com/rajeevw/ufcdata#data.csv)

 ___
## Preparando o ambiente no jupyter:

### Imports:

In [37]:
import math
import os.path
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
import random
import statsmodels.api as sm
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

### Trabalhando com os Excels:

In [38]:
data = pd.read_excel("data.xlsx")
preproc = pd.read_excel("preprocessed_data.xlsx")
fighter = pd.read_excel("raw_fighter_details.xlsx")
total = pd.read_excel("raw_total_fight_data.xlsx")
data2 = data
data.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Henry Cejudo,Marlon Moraes,Marc Goddard,2019,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0,...,2,0,0,8,Orthodox,162.56,162.56,135.0,31.0,32.0
1,Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0,...,0,2,0,5,Southpaw,165.10,167.64,125.0,32.0,31.0


In [39]:
data.weight_class = data.weight_class.astype('category')
data.Winner = data.Winner.astype('category')

In [40]:
data.Winner.value_counts()

Red     3470
Blue    1591
Draw      83
Name: Winner, dtype: int64

In [41]:
data_heavy = data.loc[(data.weight_class=='Heavyweight'),:]
bool_to_number = {False: 0, True: 1}
string_to_number = {'Blue': 0, 'Red': 1, 'Draw': 2}
data_heavy['title_bout'] = data_heavy['title_bout'].map(bool_to_number)
data_heavy['Winner'] = data_heavy['Winner'].map(string_to_number)

C:\Users\Maulem\Spyder\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Maulem\Spyder\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


#### Blue = 0
#### Red = 1

In [42]:
data_heavy.head(2)

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,Dan Miragliotta,2019,"Chicago, Illinois, USA",0,0,Heavyweight,3,0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,Jerin Valel,2019,"Ottawa, Ontario, Canada",1,0,Heavyweight,3,0,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0


In [43]:
data_heavy_util = data_heavy.drop(['Referee','date','location'], axis=1)
#dados que não se relacionam com os lutadores ou seus resultado

In [44]:
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.0,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
54,Walt Harris,Serghei Spivac,1,0,Heavyweight,3,0,0,0,NaN,...,4,0,0,4,Southpaw,195.58,195.58,250.0,24.0,35.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.0,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.0,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.0,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0


In [45]:
data_heavy_util.dropna(inplace=True)
data_heavy_util.head()

,R_fighter,B_fighter,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,...,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
4,Tai Tuivasa,Blagoy Ivanov,0,0,Heavyweight,3,0,1,0,17.00,...,2,0,0,3,Southpaw,187.96,190.50,264.0,32.0,26.0
60,Arjan Bhullar,Juan Adams,1,0,Heavyweight,3,0,1,0,8.00,...,0,0,0,2,Orthodox,185.42,190.50,245.0,27.0,32.0
63,Greg Hardy,Dmitrii Smoliakov,1,0,Heavyweight,3,2,0,0,5.00,...,0,0,0,0,Orthodox,195.58,203.20,265.0,36.0,30.0
69,Andrei Arlovski,Augusto Sakai,0,0,Heavyweight,3,0,1,0,35.00,...,9,2,0,16,Orthodox,190.50,195.58,240.0,27.0,40.0
75,Alistair Overeem,Aleksei Oleinik,1,0,Heavyweight,5,0,2,0,4.25,...,6,0,0,9,Orthodox,193.04,203.20,265.0,41.0,38.0


#### Tirando os dados categóricos:

In [46]:
categoricas = [
    'R_fighter', 
    'B_fighter', 
    'weight_class', 
    'R_Stance', 
    'B_Stance', 
]

data_heavy_cat = data_heavy_util[categoricas].astype('category')
data_heavy_num = data_heavy_util.drop(categoricas, axis=1).astype('float')

In [47]:
X = data_heavy_num.drop('Winner', axis=1)
y = data_heavy_num['Winner']

### Separando os dados em testes e treinamento

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [49]:

model = RandomForestClassifier(n_estimators=10000)

model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [50]:
#Fatores que tem o maior peso na decisão da vitória
j=1
lista_j=list()
for i, f in sorted(list(zip(model.feature_importances_, X_train.columns)), reverse=True):
    print( j,'°:',f'{f}: {i}')
    a=str(j)+'°'
    lista_j.append(a)
    j+=1


1 °: R_avg_opp_CLINCH_att: 0.016915914032218028
2 °: R_avg_opp_CLINCH_landed: 0.014885718944547944
3 °: B_avg_BODY_att: 0.013057988083284693
4 °: R_avg_opp_SIG_STR_landed: 0.013005517506540526
5 °: B_Reach_cms: 0.01279836073180058
6 °: B_avg_BODY_landed: 0.012131604481465478
7 °: R_losses: 0.012005095419889608
8 °: R_avg_opp_GROUND_att: 0.011513604043650226
9 °: R_avg_opp_TOTAL_STR_landed: 0.01151113970207975
10 °: B_avg_opp_GROUND_att: 0.011463208502131726
11 °: R_total_rounds_fought: 0.01136478164149049
12 °: R_avg_opp_GROUND_landed: 0.011240933025749066
13 °: R_avg_opp_HEAD_landed: 0.010994109126834675
14 °: B_avg_opp_GROUND_landed: 0.010467599262250361
15 °: B_avg_HEAD_att: 0.010363367830441509
16 °: R_Reach_cms: 0.010342040005542622
17 °: B_avg_TOTAL_STR_landed: 0.010277942029557745
18 °: R_avg_DISTANCE_att: 0.010274561731785458
19 °: B_avg_opp_TOTAL_STR_landed: 0.010129564089528389
20 °: B_avg_SIG_STR_att: 0.009994703451109197
21 °: B_avg_opp_LEG_att: 0.009978964907415767
22 °: B

#### Criando um dataframe com os dados acima:

In [51]:
data={'Fator':X_train.columns ,'Correlação':model.feature_importances_,}
Fator_por_corr=pd.DataFrame(data)
Fator_por_corr=Fator_por_corr.sort_values(by='Correlação', ascending=False)
Fator_por_corr['Grau de Importância']=lista_j
Fator_por_corr = Fator_por_corr.set_index('Grau de Importância')
Fator_por_corr.head()

,Fator,Correlação
Grau de Importância,,
1°,R_avg_opp_CLINCH_att,0.016916
2°,R_avg_opp_CLINCH_landed,0.014886
3°,B_avg_BODY_att,0.013058
4°,R_avg_opp_SIG_STR_landed,0.013006
5°,B_Reach_cms,0.012798


#### Com base na correlação, vamos definir a relevância:

In [52]:
def relevancia(df,coluna_nome,coluna_correlacao,acuracia):
    inuteis = []
    uteis = []
    for index,row in df.iterrows():
        if row[coluna_correlacao] >= -acuracia and row[coluna_correlacao] <= acuracia:
            inuteis.append(row[coluna_nome])
        else:
            uteis.append(row[coluna_nome])
    return uteis

In [53]:
uteis = relevancia(Fator_por_corr,'Fator','Correlação',0.01)

In [54]:
data_heavy_util_relevante = data_heavy_util.loc[:,uteis]
data_heavy_util_relevante.head()

,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,B_avg_BODY_att,R_avg_opp_SIG_STR_landed,B_Reach_cms,B_avg_BODY_landed,R_losses,R_avg_opp_GROUND_att,R_avg_opp_TOTAL_STR_landed,B_avg_opp_GROUND_att,R_total_rounds_fought,R_avg_opp_GROUND_landed,R_avg_opp_HEAD_landed,B_avg_opp_GROUND_landed,B_avg_HEAD_att,R_Reach_cms,B_avg_TOTAL_STR_landed,R_avg_DISTANCE_att,B_avg_opp_TOTAL_STR_landed
4,4.500000,3.500000,17.00,22.500000,185.42,14.500,1,7.750000,27.750000,0.000,7,2.750000,14.000000,0.00,184.5,190.50,62.0,50.750000,90.000
60,8.000000,5.333333,8.00,29.333333,203.20,8.000,1,1.333333,42.333333,2.000,8,0.666667,12.333333,2.00,131.0,190.50,121.0,33.333333,46.000
63,2.000000,1.000000,5.00,12.000000,187.96,3.500,1,3.000000,16.000000,28.500,2,3.000000,12.000000,14.50,15.5,203.20,47.0,51.000000,58.000
69,4.964286,3.035714,35.00,22.321429,195.58,21.000,12,7.964286,34.821429,0.000,59,5.357143,15.142857,0.00,169.0,195.58,125.0,54.178571,84.000
75,6.466667,4.200000,4.25,18.933333,203.20,3.875,6,4.133333,28.533333,3.625,29,3.466667,13.333333,2.75,28.5,203.20,38.5,21.666667,31.125


___
## Verificando a performance


In [55]:
X_train.columns

Index(['title_bout', 'no_of_rounds', 'B_current_lose_streak',
       'B_current_win_streak', 'B_draw', 'B_avg_BODY_att', 'B_avg_BODY_landed',
       'B_avg_CLINCH_att', 'B_avg_CLINCH_landed', 'B_avg_DISTANCE_att',
       ...
       'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Height_cms', 'R_Reach_cms',
       'R_Weight_lbs', 'B_age', 'R_age'],
      dtype='object', length=136)

In [56]:
model.feature_importances_

array([0.00039142, 0.00110902, 0.00284829, 0.00344307, 0.        ,
       0.01305799, 0.0121316 , 0.00856644, 0.00866622, 0.0093048 ,
       0.00729099, 0.00819769, 0.00834542, 0.01036337, 0.00930906,
       0.00456245, 0.00918029, 0.0089523 , 0.00634889, 0.00631558,
       0.0099947 , 0.00940169, 0.00717701, 0.00869603, 0.00928783,
       0.00608339, 0.00729766, 0.00920055, 0.01027794, 0.00452874,
       0.00458605, 0.00790066, 0.00774868, 0.00802465, 0.00728178,
       0.00672326, 0.00724857, 0.01146321, 0.0104676 , 0.00989203,
       0.00761616, 0.00391738, 0.00997896, 0.00776602, 0.00744043,
       0.00394671, 0.00913384, 0.00726346, 0.00847272, 0.00491476,
       0.00798922, 0.00736814, 0.00977277, 0.00899168, 0.01012956,
       0.00738513, 0.00775549, 0.00159093, 0.00057272, 0.00090121,
       0.00284176, 0.00510044, 0.0019866 , 0.00091363, 0.00473581,
       0.00908474, 0.01279836, 0.00895047, 0.00377492, 0.00823594,
       0.        , 0.00889014, 0.00809562, 0.00956846, 0.00884

In [57]:
y_pred = model.predict(X_test)

In [58]:
print(accuracy_score(y_test, y_pred))

0.5454545454545454


In [59]:
y_pred

array([1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1.])

In [60]:
y_test.value_counts(True)

1.0    0.621212
0.0    0.363636
2.0    0.015152
Name: Winner, dtype: float64

In [61]:
data_heavy_num.shape

(264, 137)

### Conclusão desse teste:

Tendo um acurácia de 0.6667 não é um bom resultado, visto que o modelo praticamente sempre tem como resultado o vermelho como vencedor, e como a probabilidade do lutador vermelho ganhar é de 0.6212112 ele praticamente só acerta os que o vencedor é o vermelho e erra os que o azul é o vencedor.

Assim é possível concluir que é necessário desconsiderar algumas variáveis para tentar melhorar a acurácia.

___
## Montando a Regressão Logística


## !Ainda é necessário tirar os fatores que foram julgados ter um baixo grau de importância pelo teste anterior!

In [62]:
def preparo(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp,missing='drop')
    results = model.fit()
    return results

In [63]:
Y = data_heavy_num["Winner"]
data_heavy_sem_Winner=data_heavy_num.drop('Winner',axis=1) 
X=data_heavy_sem_Winner
#np.asarray(X)

In [64]:
result = preparo(X,Y)
result.summary()

C:\Users\Maulem\Spyder\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Winner   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     1.305
Date:                Mon, 18 Nov 2019   Prob (F-statistic):             0.0635
Time:                        18:41:55   Log-Likelihood:                -100.66
No. Observations:                 264   AIC:                             439.3
Df Residuals:                     145   BIC:                             864.9
Df Model:                         118                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            4.6535      2.927      1.590      0.114      -1.131      10.438
title_bout                      -0.0643      0.192     -0.334      0.739      -0.445       0.316
no_of_rounds                     0.0958      0.062      1.533      0.127      -0.028       0.219
B_current_lose_streak           -0.2204      0.073     -3.002      0.003      -0.366      -0.075
B_current_win_streak            -0.0361      0.037     -0.983      0.327      -0.109       0.037
B_draw                        1.562e-13   4.62e-14      3.377      0.001    6.48e-14    2.48e-13
B_avg_BODY_att                  -0.0701      0.040     -1.752      0.082      -0.149       0.009
B_avg_BODY_landed                0.0788      0.049      1.623      0.107      -0.017       0.175
B_avg_CLINCH_att                 0.0259      0.025      1.025      0.307      -0.024       0.076
B_avg_CLINCH_landed             -0.0234      0.035     -0.669      0.504      -0.092       0.046
B_avg_DISTANCE_att              -0.0151      0.014     -1.059      0.291      -0.043       0.013
B_avg_DISTANCE_landed            0.0141      0.019      0.727      0.469      -0.024       0.053
B_avg_GROUND_att                -0.0245      0.023     -1.059      0.291      -0.070       0.021
B_avg_GROUND_landed              0.0242      0.032      0.764      0.446      -0.038       0.087
B_avg_HEAD_att                  -0.0283      0.022     -1.283      0.201      -0.072       0.015
B_avg_HEAD_landed                0.0352      0.028      1.266      0.208      -0.020       0.090
B_avg_KD                         0.3686      0.193      1.906      0.059      -0.014       0.751
B_avg_LEG_att                    0.0846      0.059      1.436      0.153      -0.032       0.201
B_avg_LEG_landed                -0.0990      0.071     -1.386      0.168      -0.240       0.042
B_avg_PASS                       0.1028      0.075      1.375      0.171      -0.045       0.251
B_avg_REV                        0.7377      0.295      2.497      0.014       0.154       1.322
B_avg_SIG_STR_att               -0.0137      0.023     -0.606      0.546      -0.059       0.031
B_avg_SIG_STR_landed             0.0150      0.027      0.551      0.583      -0.039       0.069
B_avg_SIG_STR_pct               -1.5196      0.575     -2.644      0.009      -2.656      -0.384
B_avg_SUB_ATT                   -0.1195      0.137     -0.871      0.385      -0.391       0.152
B_avg_TD_att                    -0.0131      0.048     -0.274      0.784      -0.107       0.081
B_avg_TD_landed                 -0.0729      0.104     -0.701      0.484      -0.278       0.133
B_avg_TD_pct                     0.3372      0.275      1.228      0.222      -0.206       0.880
B_avg_TOTAL_STR_att              0.0405      0.026      1.568      0.119    

In [65]:
'''data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]
data_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)
data_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]
data_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)'''

"data_heavy_blue_winners = data_heavy.loc[(data_heavy.Winner==0),:]\ndata_heavy_blue_winners = data_heavy_blue_winners.drop(['Winner'], axis=1)\ndata_heavy_red_winners = data_heavy.loc[(data_heavy.Winner==1),:]\ndata_heavy_red_winners = data_heavy_red_winners.drop(['Winner'], axis=1)"

In [66]:
'''blue_winners_correlation = data_heavy_blue_winners.corr()
blue_winners_correlation.head()'''

'blue_winners_correlation = data_heavy_blue_winners.corr()\nblue_winners_correlation.head()'

In [67]:
'''red_winners_correlation = data_heavy_red_winners.corr()
red_winners_correlation.head()'''

'red_winners_correlation = data_heavy_red_winners.corr()\nred_winners_correlation.head()'

___
## Concluindo

   ___
## Aperfeiçoamento:


   ___
## Referências

[Como usar a biblioteca Scikit-lear](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)

[Como funciona o Random Forest](https://towardsdatascience.com/understanding-random-forest-58381e0602d2)

[Referencia do Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

[Como funciona regressão linear](https://www.saedsayad.com/logistic_regression.htm)

[Como usar a Regressão Linear](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)



In [109]:
#criando um novo dataframe
lista_locB = ["B_fighter", "weight_class","B_Height_cms","B_Reach_cms","B_Weight_lbs","B_age"]
lista_locR = ["R_fighter", "weight_class","R_Height_cms","R_Reach_cms","R_Weight_lbs","R_age"]
data_red = data2.loc[: , lista_locR]
data_blue = data2.loc[: , lista_locB]

data_blue.columns = ["Fighter","Weight_class","Height_cms","Reach_cms","Weight_lbs","Age"]
data_red.columns = ["Fighter","Weight_class","Height_cms","Reach_cms","Weight_lbs","Age"]

data_red.head(5)

,Fighter,Weight_class,Height_cms,Reach_cms,Weight_lbs,Age
0,Henry Cejudo,Bantamweight,162.56,162.56,135.0,32.0
1,Valentina Shevchenko,Women's Flyweight,165.10,167.64,125.0,31.0
2,Tony Ferguson,Lightweight,180.34,193.04,155.0,35.0
3,Jimmie Rivera,Bantamweight,162.56,172.72,135.0,29.0
4,Tai Tuivasa,Heavyweight,187.96,190.50,264.0,26.0


In [100]:
indice = data2.set_index('R_fighter')
indice

,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_opp_KD,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_opp_PASS,B_avg_opp_REV,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_opp_SIG_STR_pct,B_avg_opp_SUB_ATT,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_total_rounds_fought,B_total_time_fought(seconds),B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_BODY_att,R_avg_BODY_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_KD,R_avg_LEG_att,R_avg_LEG_landed,R_avg_PASS,R_avg_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_att,R_avg_TD_landed,R_avg_TD_pct,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_longest_win_streak,R_losses,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_opp_KD,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_opp_PASS,R_avg_opp_REV,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_opp_SIG_STR_pct,R_avg_opp_SUB_ATT,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_opp_TD_pct,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_total_rounds_fought,R_total_time_fought(seconds),R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
R_fighter,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Henry Cejudo,Marlon Moraes,Marc Goddard,2019,"Chicago, Illinois, USA",Red,True,Bantamweight,5,0,4,0,9.200000,6.000000,0.200000,0.000000,62.600000,20.600000,2.600000,2.000000,48.600000,11.200000,0.800000,7.600000,5.400000,0.400000,0.000000,65.400000,22.600000,0.466000,0.400000,0.800000,0.200000,0.100000,66.400000,23.600000,4,1,6.400000,4.000000,1.000000,0.600000,51.200000,17.400000,0.600000,0.200000,39.600000,9.400000,0.200000,6.800000,4.800000,0.000000,0.000000,52.800000,18.200000,0.236000,0.000000,1.000000,0.400000,0.100000,53.800000,19.200000,9,419.400000,0,0,1,0,2,1,0,4,Orthodox,167.64,170.18,135.0,0,4,0,21.900000,16.400000,17.000000,11.000000,75.000000,26.500000,9.400000,6.500000,74.200000,23.900000,0.400000,5.300000,3.700000,1.200000,0.000000,101.400000,44.000000,0.466000,0.100000,5.300000,1.900000,0.458000,129.900000,69.100000,4,2,13.300000,8.800000,7.500000,5.100000,90.500000,26.800000,0.800000,0.300000,76.100000,17.300000,0.100000,9.400000,6.100000,0.000000,0.000000,98.800000,32.200000,0.336000,0.000000,0.900000,0.100000,0.050000,110.500000,43.300000,27,742.600000,3,0,2,4,2,0,0,8,Orthodox,162.56,162.56,135.0,31.0,32.0
Valentina Shevchenko,Jessica Eye,Robert Madrigal,2019,"Chicago, Illinois, USA",Red,True,Women's Flyweight,5,0,3,0,14.600000,9.10000

In [99]:
pd.set_option('display.max_columns', 500)
testando = indice.loc["Royce Gracie", :]
testando.head(5)

,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_opp_KD,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_opp_PASS,B_avg_opp_REV,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_opp_SIG_STR_pct,B_avg_opp_SUB_ATT,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_total_rounds_fought,B_total_time_fought(seconds),B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_BODY_att,R_avg_BODY_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_KD,R_avg_LEG_att,R_avg_LEG_landed,R_avg_PASS,R_avg_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_att,R_avg_TD_landed,R_avg_TD_pct,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,R_longest_win_streak,R_losses,R_avg_opp_BODY_att,R_avg_opp_BODY_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_opp_HEAD_att,R_avg_opp_HEAD_landed,R_avg_opp_KD,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_opp_PASS,R_avg_opp_REV,R_avg_opp_SIG_STR_att,R_avg_opp_SIG_STR_landed,R_avg_opp_SIG_STR_pct,R_avg_opp_SUB_ATT,R_avg_opp_TD_att,R_avg_opp_TD_landed,R_avg_opp_TD_pct,R_avg_opp_TOTAL_STR_att,R_avg_opp_TOTAL_STR_landed,R_total_rounds_fought,R_total_time_fought(seconds),R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
R_fighter,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Royce Gracie,Dan Severn,John McCarthy,1994,"Tulsa, Oklahoma, USA",Red,True,Catch Weight,1,0,2,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,1.50,0.0,0.0,0.00,0.0000,1.0,3.00,1.5,0.7,0.0,0.00,2,0,2.0,1.5,1.5,1.5,2.5,1.50,1.50,1.50,2.0,2.00,0.0,1.50,1.00,0.0,0.0,5.50,4.5,0.69,0.0,0.00,0.00,0.000,8.50,7.0,2,78.5,0,0,0,0,0,2,0,2,Southpaw,187.96,NaN,250.0,0,10,0,1.000000,0.500000,2.000000,1.200000,1.900000,0.400000,1.400000,1.100000,2.900000,1.700000,0.0,1.400000,0.500000,1.500000,0.300000,5.300000,2.700000,0.343000,1.100000,1.500000,0.600000,0.433000,25.800000,22.900000,10,0,0.500000,0.400000,0.800000,0.600000,0.700000,0.1,1.800000,0.700000,2.600000,0.900000,0.0,0.200000,0.100000,0.200000,0.100000,3.300000,1.400000,0.188000,0.0,0.300000,0.200000,0.200000,4.700000,2.700000,10,168.300000,2,0,0,0,1,9,0,10,Southpaw,185.42,NaN,175.0,36.0,28.0
Royce Gracie,Keith Hackney,John McCarthy,1994,"Tulsa, Oklahoma, USA",Red,False,Open Weight,1,0,2,0,0.50,0.00,1.50,1.00,7.50,4.0,23.0,16.0,25.0,15.0,0.5,6.50,6.0,0.50,0.0,32.0,21.00,0.6250,0.5,0.00,0.0,0.0,34.0,23.00,2,0,0.0,0.0,0.0,0.0,1.0,0.00,1.50,0.50,2.5,0.50,0.0,0.00,0.00,0.5,0.0,2.50,0.5,0.10,0.5,1.00,0.50,0.500,6.50,4.0,2,141.5,0,0,0,0,1,1,0,2,Sideways,180.34,NaN,200.0,0,9,0,0.555556,0.222222,0.777778,0.3